In [1]:
import os
import logging, timeit
from btEngine2.DataLoader import DataLoader
from btEngine2.MarketData import MarketData
from btEngine2.TradingRule import TradingRule



# Define paths and configurations for DataLoader
ticker_csv_path = r'G:\Projects\BackTesting1.0\Data\Inputs\TickerList-Futs.csv'
save_directory = r"G:\Projects\BackTesting1.0\Data\Bloomberg\Futures"
helper_directory = r'G:\Projects\BackTesting1.0\Data\Bloomberg\HelperFiles'

bt_folder = r'BackTests\ratioRSI_resrch'

# Define paths to auxiliary data for MarketData
tick_values_path = os.path.join(helper_directory, 'fut_val_pt.parquet')
fx_rates_path = os.path.join(helper_directory, 'fxHist.parquet')

# Initialize the MarketData
market_data = MarketData(
    base_directory=save_directory,
    tick_values_path=tick_values_path,
    fx_rates_path=fx_rates_path,
    instrument_type="Futures",
    n_threads=8,  # Number of threads for parallel data loading
    log_level=logging.ERROR  # Set to DEBUG for more detailed logs
)


In [2]:
tick = 'XTD1 Curncy'
# Access data for a specific ticker
try:
    test_df = market_data.get_ticker_data(tick)
    print(test_df)
except ValueError as e:
    print(e)

# Access all preprocessed data
all_data = market_data.get_data()
print(f"Total tickers loaded: {len(all_data)}")

# Access FX rates
fx_rates = market_data.get_fx_rates()
# Access tick values
tick_values = market_data.get_tick_values()
# Access asset classes
asset_classes = market_data.get_asset_classes()

#market_data = market_data.date_filter(start_date='01012010')

shape: (2_378, 14)
┌────────────┬────────┬────────┬────────┬───┬─────────┬─────────┬─────────────────┬────────────────┐
│ Date       ┆ Open   ┆ High   ┆ Low    ┆ … ┆ BadOHLC ┆ FX_Rate ┆ Tick_Value_Base ┆ Tick_Value_USD │
│ ---        ┆ ---    ┆ ---    ┆ ---    ┆   ┆ ---     ┆ ---     ┆ ---             ┆ ---            │
│ date       ┆ f64    ┆ f64    ┆ f64    ┆   ┆ bool    ┆ f64     ┆ f64             ┆ f64            │
╞════════════╪════════╪════════╪════════╪═══╪═════════╪═════════╪═════════════════╪════════════════╡
│ 2015-09-01 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-02 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-03 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-04 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000.0        ┆ 100000.0       │
│ 2015-09-07 ┆ null   ┆ null   ┆ null   ┆ … ┆ false   ┆ 1.0     ┆ 100000

In [3]:

pSizeParams = {
    'AssetVol': 5_000_000,  # Target asset volatility in USD
    'VolLookBack': 30,
    'VolMethod': 'ewm'  # Lookback period for volatility calculation
}


In [4]:
from btEngine2.Rules.MeanReversion.ratioMR import *

fi_asst = 'WN1 Comdty'
# Define the list of asset pairs you want to trade
pairs = [
    ('NQ1 Index', fi_asst),
    (fi_asst, 'NQ1 Index')
]
    
strategy_params2 = {
    'pairs': pairs,
    'N': 5,
    'rsi_period': 3,
    'rsi_threshold': 10.0,
    'market_data': market_data,  # Pass the MarketData instance
}

   
strategy_params_short = {
    'pairs': pairs,
    'N': 5,
    'rsi_period': 3,
    'rsi_threshold': 10.0,
    'market_data': market_data,  # Pass the MarketData instance
}
# Set strategy parameters
strategy_params = {
    'pairs': pairs,
    'N': 5,
    'rsi_period': 3,
    'rsi_threshold': 10.0,
    'lmt_order': True,
    'lmt_day': 2,
    'lmt_day_only': False,
    'lmt_atr': 1,
    'lmt_epsilon': 0.1,
    'atr_period': 5,
    'atr_type': 'atr',
    'market_data': market_data  # Pass the MarketData instance
}

strategy_params3 = strategy_params2.copy()
strategy_params3['trend_filter'] = 120

strategy_params4 = strategy_params2.copy()
strategy_params4['oversold_cond'] = True

strategy_params5 = strategy_params3.copy()
strategy_params5['oversold_cond'] = True


In [5]:
### Where the magic happens

asst_main = 'NQ1 Index'

cmp_assts = market_data.get_asset_classes()['eq-us']
cmp_assts
cmp_assts = ['US1 Comdty', 'WN1 Comdty']


respnls = {}
results = {}
statsdf = {}
trules = {}




for asst in cmp_assts:
    
    pairs = [
        (asst_main, asst),
        (asst, asst_main)
    ]
    
    strategy_params2 = {
        'pairs': pairs,
        'N': (5, 2),
        'rsi_period': (3,3),
        'rsi_threshold': (10.0,8.0),
        'lmt_order': False,
        'lmt_day': 1,
        'lmt_day_only': True,
        'lmt_atr': -0.25,
        'lmt_epsilon': 0.1,
        'atr_period': 3,
        'atr_type': 'atr',
        'market_data': market_data,
        'risk_ratio': (1.0, 1.1)
        
    }

    tmp = TradingRule(
        market_data=market_data,
        trading_rule_function=ratioMR_rsi_long,
        trading_params=strategy_params2,
        position_sizing_params=pSizeParams,  # Define as needed
        incl_assets=[asst_main, asst],  # Include all involved assets
        name_label='TestMR_' + asst,
        strat_descr= f'rsi mr strategy {asst_main} vs. {asst}',
        log_level=logging.ERROR,
        bt_folder = bt_folder
    )

    res = tmp.backtest_all_assets(save=True)
    resdb = tmp.plot_equity(byassets=True, totalsys=True, start_date = '01012000', save_fig=True)
    stats = tmp.calculate_statistics(byassets=True, totalsys=True, start_date='01012000')
    statsdf[asst] = stats
    results[asst] = resdb
    respnls[asst] = res
    trules[asst] = tmp

# Extract the desired statistics from each dataframe in statsdf

required_columns = ['Average Ann. PnL', 'Average Ann. Vol', 'Sharpe Ratio (ann.)', 'Sortino Ratio (ann.)', 'Hit Rate (%)', 'Profit Factor', 
                    'Max drawdown', 'Average drawdown', 'Avg Drawdown Duration', 'Drawdowns per year', 'Max Drawdown Duration']
extracted_stats = {key: statsdf[key].loc['Total', [col for col in required_columns if col in statsdf[key].columns]] for key in statsdf.keys()}

# Convert the dictionary to a dataframe
extracted_stats_df = pd.DataFrame.from_dict(extracted_stats, orient='index')
extracted_stats_df.to_clipboard()
extracted_stats_df

,Average Ann. PnL,Average Ann. Vol,Sharpe Ratio (ann.),Sortino Ratio (ann.),Hit Rate (%),Profit Factor,Max drawdown,Average drawdown,Avg Drawdown Duration,Drawdowns per year
US1 Comdty,2.639843e+06,3.242956e+06,0.814024,0.641857,59.554140,1.392255,8201776.250,1.673970e+06,37.278571,5.589354
WN1 Comdty,4.262715e+06,3.097123e+06,1.376347,1.010579,64.044944,1.760658,3934944.375,9.756537e+05,24.537037,7.304348


In [13]:
testtr = trules['US1 Comdty']

testtr.perf_table(byac=False,byassets=True, table_detail='full')

,Period,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,2007,2006,2005,2004,2003,2002,2001,2000,1999,Total
AssetClass,Asset,,,,,,,,,,,,,,,,,,,,,
Unknown,Total,6040658.75,5008267.5,3767868.75,7865151.875,-52528.125,653591.25,1289110.0,8940641.875,3252635.625,1518508.125,...,1834280.625,-4158.125,1692100.625,-469927.5,4448416.25,-566613.125,-1507578.125,588146.875,1833593.75,67955368.75
eq-us,NQ1 Index,5743040.00,4150955.0,2903750.00,6546630.000,594100.000,-7990.00,409385.0,4905395.000,704795.000,296855.000,...,3125990.000,400195.000,-905790.000,-70490.0,3339960.00,-1657710.000,-147050.000,1990200.000,1015400.00,50179850.00
fi-us,US1 Comdty,297618.75,857312.5,864118.75,1318521.875,-646628.125,661581.25,879725.0,4035246.875,2547840.625,1221653.125,...,-1291709.375,-404353.125,2597890.625,-399437.5,1108456.25,1091096.875,-1360528.125,-1402053.125,818193.75,17775518.75


In [7]:
respnls['US1 Comdty'][0]['US1 Comdty'].to_clipboard()

In [8]:
nq_pl = market_data.get_ticker_data('WN1 Comdty')
nq = nq_pl.to_pandas() 
nq.set_index('Date', inplace=True)

In [ ]:
import plotly.graph_objects as go

# Extract the OHLC data for 'NQ1 Index'
nq_data = nq[['Open', 'High', 'Low', 'Close']]

# Create the OHLC plot
fig = go.Figure(data=[go.Ohlc(
    x=nq_data.index,
    open=nq_data['Open'],
    high=nq_data['High'],
    low=nq_data['Low'],
    close=nq_data['Close'],
    name='NQ1 Index'
)])

# Update layout for better visualization
fig.update_layout(
    title='NQ1 Index OHLC Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()

In [ ]:
mthlypnl = trules['WN1 Comdty'].monthly_pnl(filter_assets=['WN1 Comdty'],save=True, name='WN monthlypnl.csv')
mthlypnl = trules['WN1 Comdty'].monthly_pnl(filter_assets=['NQ1 Index'],save=True, name='NQ monthlypnl.csv')
mthlypnl = trules['WN1 Comdty'].monthly_pnl(save=True, name='Strategy monthlypnl.csv')

In [ ]:
mthlypnl

In [ ]:
statsdf = trules['WN1 Comdty'].calculate_statistics(byac=True, totalsys=True, start_date='01012000', save=True)

In [ ]:
statsdf

In [14]:
trades_list = trules['US1 Comdty'].tradeslist

In [ ]:
trades_list
import matplotlib.pyplot as plt
# Get unique assets from the trades list
unique_assets = trades_list['Asset'].unique()

# Plot histogram for each unique asset
for asset in unique_assets:
    asset_trades = trades_list[trades_list['Asset'] == asset]
    plt.figure(figsize=(10, 6))
    plt.hist(asset_trades['Total PnL'], bins=50, edgecolor='black')
    plt.title(f'Histogram of Total PnL for {asset}')
    plt.xlabel('Total PnL')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()


In [ ]:
# Plot histogram of 'Total PnL' column
plt.figure(figsize=(10, 6))
plt.hist(trades_list['Total PnL'], bins=50, edgecolor='black')
plt.title('Histogram of Total PnL')
plt.xlabel('Total PnL')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()